### Context Managers

try...finally...

The finally section of a try *always* executes

try:  
...  
except:  
...  
finally:  
...

The finally always executes, even if an exception occurs in the except block

This works even if inside a function and a return is in the try or except blocks

This is very useful for writing code that should execute no matter what happens

But this can get cumbersome...is there a better way?

#### Pattern

- create some object  
- do some work with that object
- clean up the object after we're done using it

We want to make this easy, can do this by automating the cleanup

Context Managers - PEP 343

In [ ]:
with context as obj_name: #obj_name is something return from context (optional)
    # with block (can use obj_name)
# after the with block, context is clean up automatically

Example:

In [ ]:
with open(file_name) as f: # enter the context   (optional) an object is returned
    #file is now open
                           # exit the context
# file is now closed

#### The context management protocol

Classes implement the context management protocol by implementing two methods:  
- \_\_enter\_\_   (setup, optionally return some object)
- \_\_exit\_\_   (tear down / cleanup)

for example:


In [ ]:
with CtxManager() as obj:
    # do something
# done with context

This does the following behind the scenes...

mgr = CtxManager()  
obj = mgr.\_\_enter__()  
try:  
\# do something  
finally:  
\# done with context  
mgr.\_\_exit__()

(note, the above pseudocode is over-simplified for the purpose of the example, there is no exception handling present)

#### Use Cases

Very common usage is for opening a file (creating a resource) and closing the file (releasing resource)  
But context managesr can be used for much more than creating and releasing resources

##### Common Patterns:
- Open-Close
- Lock-Release
- Change-Reset
- Start-Stop
- Enter-Exit

ie:
- file context managers
- decimal contexts

#### How Context Protocol Works

In [ ]:
class MyClass:
    def __init__(self):
        # init class
    
    def __enter__(self):
        return obj
    
    def __exit__(self, '''+ ...'''):
        # clean up obj

It works in conjunction with a **with** statement

In [ ]:
my_obj = MyClass()

The above works as a regular class, and \_\_enter\_\_, \_\_exit\_\_ were not called

In [ ]:
with MyClass as obj:

- This creates an instance of MyClass
- Theres no associated symbol, but an instance exists -> lets call it *my_instance*
- my_instance.\_\_enter\_\_() is called
- the return value from \_\_enter\_\_ is assigned to obj (**not** the instance of *MyClass* that was created)

after the with block, or if an exception occurs inside the with block:
- my_instance.\_\_exit\_\_ is called

#### Scope of with block

The **with** block is not like a function or a comprehension  
The scope of anything in the with block (including the object returned from \_\_enter\_\_) is the same scope as the with statement itself

ie:

In [ ]:
# module.py
with open(fname) as f: # f is a symbol in global scope
    row = next(f) # row is also in the global scope
    
print(f) # f is closed, but the symbol exists
print(row) # row is available and has a value

#### The \_\_enter__ method

In [ ]:
def __enter__(self):

This method should perform whatever setup it needs to  
It can optionally return an object -> as returned_obj

That's all there is to this method.

#### The \_\_exit__ method

This is a little more complicated...

Remember the finally in a try statement? -> always runs even if an exception occurs  
\_\_exit__ is similar -> runs even if an exception occurs in with block  
But should it handle things differently if an exception occured?  
-> maybe... -> so it needs to know about any exceptions that occured -> it also needs to tell Python whether to silence the exception, or let it propagate

The \_\_exit__ Method

In [ ]:
with MyContext() as obj:
    raise ValueError
    
print('done')

**Scenario 1**  
\_\_exit__ receives error, performs some clean up and silences error  
print statement runs
no exception is seen

**Scenario 2**  
\_\_exit__ receives error, performs some clean up and let's error propogate
print statement does not run
the ValueException is seen

Therefore, the exit method needs 3 arguements:
- the exception type that occure (if any, None otherwise)
- the exception value that occured (if any, None otherwise)
- the traceback object if an exception occured (if any, None otherwise)

It also returns True or False:
- True = silence any raised exception
- False = do no silence a raised exception

ie:

In [ ]:
def __Exit__(self, exc_type, exc_value, exc_trace):
    # do clean up work here
    return True # or False

#### Code Examples:

In [5]:
try:
    10 / 2
except ZeroDivisionError:
    print('Zero division exception occured')
finally:
    print('finally ran!')

finally ran!


In [7]:
try:
    10 / 0
except ZeroDivisionError:
    print('Zero division exception occured')
finally:
    print('finally ran!')

Zero division exception occured
finally ran!


In [10]:
def my_func():
    try:
        10 / 0
    except ZeroDivisionError:
        return 
    finally:
        print('finally ran!')

In [11]:
my_func()

finally ran!


In [12]:
try:
    print('Opening file...')
    f = open('test.txt', 'w')
    a = 1 / 0
except:
    print('an exception occured')
finally:
    print('Closing file...')
    f.close()

Opening file...
an exception occured
Closing file...


In [13]:
try:
    print('Opening file...')
    f = open('test.txt', 'w')
    a = 1 / 0
finally:
    print('Closing file...')
    f.close()

Opening file...
Closing file...


ZeroDivisionError: division by zero

In [14]:
with open('test.txt', 'w') as file:
    print('inside with: file closed?', file.closed)
    
print('after with: file closed?', file.closed)

inside with: file closed? False
after with: file closed? True


In [15]:
def test():
    with open('test.txt', 'w') as file:
        print('inside with: file closed?', file.closed)
        return file
        print('here - will never run')

In [16]:
file = test()

inside with: file closed? False


In [17]:
file.closed

True

In [18]:
with open('test.txt', 'w') as file:
    print('inside with: file closed?', file.closed)
    raise ValueError()

inside with: file closed? False


ValueError: 

In [19]:
file.closed

True

In [20]:
with open('test.txt', 'w') as f:
    f.writelines('this is a test')

In [21]:
with open('test.txt') as f:
    row = next(f)

In [22]:
f.closed

True

In [23]:
row

'this is a test'

In [28]:
class MyContext:
    def __init__(self):
        print('__init__ running...')
        self.obj = None
        
    def __enter__(self):
        print('entering context...')
        self.obj = 'the Return object'
        return self.obj
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        if exc_type:
            print(f'*** Error occured: {exc_type}, {exc_value}')
            return False

In [29]:
with MyContext() as obj:
    print('inside with block')
    raise ValueError('custom message')

__init__ running...
entering context...
inside with block
exiting context...
*** Error occured: <class 'ValueError'>, custom message


ValueError: custom message

In [31]:
ctx = MyContext()
print('created context...')
with ctx as obj:
    print('inside with block', obj)
    raise ValueError('custom message')

__init__ running...
created context...
entering context...
inside with block the Return object
exiting context...
*** Error occured: <class 'ValueError'>, custom message


ValueError: custom message

In [32]:
class MyContext:
    def __init__(self):
        print('__init__ running...')
        self.obj = None
        
    def __enter__(self):
        print('entering context...')
        self.obj = 'the Return object'
        return self.obj
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context...')
        if exc_type:
            print(f'*** Error occured: {exc_type}, {exc_value}')
            return True

In [33]:
ctx = MyContext()
print('created context...')
with ctx as obj:
    print('inside with block', obj)
    raise ValueError('custom message')

__init__ running...
created context...
entering context...
inside with block the Return object
exiting context...
*** Error occured: <class 'ValueError'>, custom message


In [34]:
obj

'the Return object'

In [35]:
class Resource:
    def __init__(self, name):
        self.name = name
        self.state = None

In [40]:
class ResourceManager:
    def __init__(self, name):
        self.name = name
        self.resource = None
        
    def __enter__(self):
        print('entering context')
        self.resource = Resource(self.name)
        self.resource.state = 'created'
        return self.resource
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('exiting context')
        self.resource.state = 'destroyed'
        if exc_type:
            print('error occured')
        return False

In [41]:
with ResourceManager('spam') as res:
    print(f'{res.name} = {res.state}')
print(f'{res.name} = {res.state}')

entering context
spam = created
exiting context
spam = destroyed


In [42]:
'res' in globals()

True

In [45]:
class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opneing file...')
        self.file = open(self.name, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('closing file...')
        self.file.close()
        return False

In [46]:
with File('test.txt', 'w') as f:
    f.write('This is a late parrot!')

opneing file...
closing file...


In [47]:
with File('test.txt', 'r') as f:
    print(f.readlines())

opneing file...
['This is a late parrot!']
closing file...


In [48]:
def test():
    with File('test.txt', 'w') as f:
        f.write('This is a late parrot!')
        return f
    print(f.closed)

In [49]:
f = test()

opneing file...
closing file...


In [50]:
class File:
    def __init__(self, name, mode):
        self.name = name
        self.mode = mode
        
    def __enter__(self):
        print('opneing file...')
        self.file = open(self.name, self.mode)
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        print('closing file...')
        self.file.close()
        return False

In [51]:
with File('test.txt', 'r') as file_ctx:
    print(next(file_ctx.file))
    print(file_ctx.name)
    print(file_ctx.mode)

opneing file...
This is a late parrot!
test.txt
r
closing file...
